# Fashion AI Training - Google Colab 版本 (修復依賴衝突)

## 🔧 專為解決 Colab 依賴衝突設計

此 Notebook 會自動處理 `sentence-transformers` 和 `transformers` 版本衝突問題。

### 📋 使用步驟:
1. 按順序執行每個 cell
2. 在依賴安裝完成後重新啟動運行時
3. 上傳訓練圖片
4. 開始訓練

### 💡 提示:
- 確保已啟用 GPU (Runtime > Change runtime type > GPU)
- 建議使用 T4 或更高等級的 GPU
- 訓練時間約 30-60 分鐘

## 🔧 步驟 1: 修復依賴衝突

**重要**: 執行此 cell 後請重新啟動運行時 (Runtime > Restart runtime)

In [ ]:
# 修復 Colab 依賴衝突
print("🔧 正在修復 Google Colab 依賴衝突...")

# 1. 卸載衝突套件
!pip uninstall -y sentence-transformers transformers

# 2. 安裝兼容版本
!pip install transformers>=4.41.0 --force-reinstall

# 3. 安裝核心套件
!pip install diffusers[torch] accelerate peft packaging

# 4. 重新安裝 sentence-transformers
!pip install sentence-transformers

# 5. 嘗試安裝 xformers (可選)
!pip install xformers --index-url https://download.pytorch.org/whl/cu118

print("✅ 依賴修復完成！")
print("🔄 請重新啟動運行時 (Runtime > Restart runtime)，然後執行下一個 cell")

## ✅ 步驟 2: 檢查安裝

**重新啟動運行時後**，執行此 cell 檢查套件是否正確安裝

In [ ]:
# 檢查套件版本
import torch
import transformers
import diffusers

print("📋 套件版本檢查:")
print(f"   torch: {torch.__version__}")
print(f"   transformers: {transformers.__version__}")
print(f"   diffusers: {diffusers.__version__}")

# 檢查 GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n🔧 GPU: {gpu_name}")
    print(f"💾 VRAM: {gpu_memory:.1f} GB")
else:
    print("❌ 沒有可用的 GPU")

# 測試導入
try:
    from diffusers import StableDiffusionPipeline
    from peft import LoraConfig
    print("\n✅ 所有套件導入成功")
except ImportError as e:
    print(f"❌ 導入失敗: {e}")

## 📁 步驟 3: 掛載 Google Drive

In [ ]:
from google.colab import drive
import os

# 掛載 Google Drive
drive.mount('/content/drive')

# 創建工作目錄
work_dir = "/content/drive/MyDrive/fashion_ai_training"
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"📁 工作目錄: {work_dir}")
print("✅ Google Drive 已掛載")

## 📤 步驟 4: 上傳訓練圖片

In [ ]:
from google.colab import files
import os

print("📤 請上傳訓練圖片 (支援 JPG, PNG 格式)...")
uploaded = files.upload()

# 處理上傳的圖片
image_paths = []
upload_dir = "/content/uploaded_images"
os.makedirs(upload_dir, exist_ok=True)

for filename, content in uploaded.items():
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        file_path = os.path.join(upload_dir, filename)
        with open(file_path, 'wb') as f:
            f.write(content)
        image_paths.append(file_path)
        print(f"✅ 已處理: {filename}")

print(f"\n📊 總共上傳了 {len(image_paths)} 張圖片")

# 顯示圖片預覽
if image_paths:
    from PIL import Image
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, min(3, len(image_paths)), figsize=(12, 4))
    if len(image_paths) == 1:
        axes = [axes]
    
    for i, img_path in enumerate(image_paths[:3]):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(os.path.basename(img_path))
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 沒有有效的圖片檔案")

## 🚀 步驟 5: 開始訓練

這個 cell 會執行實際的 Fashion AI 訓練。根據 GPU 類型和圖片數量，預計需要 30-60 分鐘。

In [ ]:
# 載入完整的訓練腳本
exec(open('/content/colab_training_fixed.py').read())

# 或者手動執行簡化版本
print("🎨 Fashion AI Training 開始...")
print("💡 由於這是簡化版本，完整訓練請上傳完整的腳本檔案")
print("✅ 環境已準備就緒，可以開始訓練！")

## 📦 步驟 6: 下載結果

訓練完成後，執行此 cell 來打包和下載結果。

In [ ]:
import zipfile
from datetime import datetime
from google.colab import files

# 創建下載包
package_name = f"fashion_ai_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"

with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # 添加訓練結果 (如果存在)
    result_dirs = ['models', 'validation', 'checkpoints']
    
    for dir_name in result_dirs:
        if os.path.exists(dir_name):
            for root, dirs, files in os.walk(dir_name):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)
    
    # 添加 README
    readme_content = f"""Fashion AI Training Results
訓練日期: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}
圖片數量: {len(image_paths) if 'image_paths' in locals() else 0}

使用方法:
1. 解壓縮檔案
2. 將 LoRA 權重載入到 Stable Diffusion WebUI
3. 開始生成時尚圖片
"""
    
    with open('README.txt', 'w', encoding='utf-8') as f:
        f.write(readme_content)
    zipf.write('README.txt')

print(f"📦 打包完成: {package_name}")

# 下載結果
files.download(package_name)

print("🎉 訓練完成！結果已下載到您的電腦")

## 🔧 故障排除

### 常見問題:

1. **依賴衝突**: 重新啟動運行時並重新執行依賴安裝
2. **GPU 記憶體不足**: 減少批次大小或使用更小的模型
3. **訓練中斷**: 檢查點會自動保存，可以從中斷處恢復

### 手動修復命令:

```bash
# 清理環境
!pip uninstall -y sentence-transformers transformers

# 重新安裝
!pip install transformers>=4.41.0 --force-reinstall
!pip install diffusers[torch] accelerate peft
```

### 聯繫支援:
如果問題持續存在，請檢查 GitHub 倉庫的 Issues 部分。